# 1. 데이터 불러오기

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('./housing/train.csv')

In [3]:
print(f'train set은 {train.shape[1]}개의 feature를 가진 {train.shape[0]}개의 데이터 샘플로 이루어져 있습니다.')

train set은 15개의 feature를 가진 1350개의 데이터 샘플로 이루어져 있습니다.


### train.shape?
**pandas.DataFrame.shape**는 파일의 행과 열의 갯수를 알려준다

In [6]:
train.head()

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
0,1,10,2392,Ex,3,968,Ex,2392,2392,Ex,2,2003,2003,2003,386250
1,2,7,1352,Gd,2,466,Gd,1352,1352,Ex,2,2006,2007,2006,194000
2,3,5,900,TA,1,288,TA,864,900,TA,1,1967,1967,1967,123000
3,4,5,1174,TA,2,576,Gd,680,680,TA,1,1900,2006,2000,135000
4,5,7,1958,Gd,3,936,Gd,1026,1026,Gd,2,2005,2005,2005,250000


## 결측치 확인하기

In [9]:
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns: #파일의 각 column을 확인
        missing_values = sum(dataframe[col].isna()) #각 column에 NA의 개수를 합해서 missing_values에 저장
        is_missing = True if missing_values != 0 else False #missing_values의 값이 0이 아니라면, 그 column의 is_missing 값은 'True'
        if is_missing: #is_missing이 'True'라면
            print(f"결측치가 있는 컬럼은: {col}입니다.")
            print(f"해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.")
            missing_col.append([col, dataframe[col].dtype]) #이 줄은 이해가 잘 가지않는다
    if missing_col == []:
        print("결측치가 존재하지 않습니다.")
    return missing_col


In [10]:
missing_col = check_missing_col(train)

결측치가 존재하지 않습니다.


# 2. 데이터 설명 방정식 만들기

## Y = aX + b
X 는 주어진 데이터, Y 는 예측하려는 데이터를 뜻한다.
우리의 목표는 X에 어떤 a를 곱하고, 어떤 b를 더해야 정확하게 Y를 구할 수 있는지, 그 a와 b를 찾는 것!

먼저, X = {[Gr Liv Area]}를 이용해서 
Y = {[SalePrice]}를 예측해보자

In [13]:
#feature를 확인힙니다
train.columns

Index(['id', 'Overall Qual', 'Gr Liv Area', 'Exter Qual', 'Garage Cars',
       'Garage Area', 'Kitchen Qual', 'Total Bsmt SF', '1st Flr SF',
       'Bsmt Qual', 'Full Bath', 'Year Built', 'Year Remod/Add',
       'Garage Yr Blt', 'target'],
      dtype='object')

In [16]:
# X와 Y를 각각 따로 저장합니다
x_feature = 'Gr Liv Area'
y_feature = 'target'

X = train[x_feature]
Y = train[y_feature]

X

0       2392
1       1352
2        900
3       1174
4       1958
        ... 
1345    1756
1346    2748
1347    1214
1348     894
1349     907
Name: Gr Liv Area, Length: 1350, dtype: int64

### a 구하기
먼저, X와 Y가 어떤 관계를 갖는지, 그 평균 값을 구해보자

In [20]:
import numpy as np

#각자의 평균을 구합니다.
X_mean = np.mean(X)
Y_mean = np.mean(Y)

#평균값을 이용해서 관계를 구합니다.
relation = Y_mean/X_mean

print(f"{x_feature}의 평균: {X_mean:.2f}\n{y_feature}의 평균: {Y_mean:.2f}")
print(f"따라서 {y_feature}는 {x_feature}의 {relation:.2f}배 입니다.")
print(f"즉, a = {relation:.2f} 입니다.")
a = round(relation, 2)
print(f"업데이트된 방정식: Y = {a:.2f} * X + b")

Gr Liv Area의 평균: 1513.54
target의 평균: 186406.31
따라서 target는 Gr Liv Area의 123.16배 입니다.
즉, a = 123.16 입니다.
업데이트된 방정식: Y = 123.16 * X + b


### b 구하기
a는 X와 Y의 강력한 관계를 나타냈다면, b는 그보다는 약한 관계를 나타냅니다.  
a만으로는 설명하지 못하는 부분을 구하는 과정이라고 볼 수 있습니다.  
a를 넣어서 새롭게 업데이트 된 방정식을 이용하면 b를 구할 수 있습니다.  

In [23]:
# b = Y - a * X 를 그대로 작성합니다. 여기서는 평균값을 이용합니다
b = Y_mean - a * X_mean
b = round(b, 2)

print(f"b는 {b:.2f}입니다.")
print(f"업데이트 된 방정식: Y = {a:.2f} * X + ({b:.2f})")

b는 -1.55입니다.
업데이트 된 방정식: Y = 123.16 * X + (-1.55)


In [24]:
predict_Y = a * X + b
predict_Y

0       294597.17
1       166510.77
2       110842.45
3       144588.29
4       241145.73
          ...    
1345    216267.41
1346    338442.13
1347    149514.69
1348    110103.49
1349    111704.57
Name: Gr Liv Area, Length: 1350, dtype: float64

# 3. 예측 결과 평가하기
## NMAE
NMAE란? 정규화된 오차의 평균의 평균.  
다시 말하자면, 정규화된(Normalized) 실제 값과 예측 값의 차이(Error)를 절대값(Absolute)으로 변환해 평균화(Mean)한 것.

In [26]:
# 대회 규칙의 평가 산식 함수를 그대로 사용합니다.
def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true)) 
    return score

In [28]:
#실제 정답 값 준비
real_answer = Y.copy()

#정답과 예측 값을 함수에 넣어 결과를 확인
error = NMAE(real_answer, predict_Y)

print(f"이 방정식의 평균 에러는 {error:.2f}입니다.")

이 방정식의 평균 에러는 0.20입니다.


# 4. test_data 예측하기
X로 Y를 예측하는 방정식을 이용해서 실제로 주어지지 않은 데이터를 예측

In [29]:
# 우리가 예측하고자 하는 test 데이터를 불러옵니다
test = pd.read_csv("./housing/test.csv")

test.head()

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt
0,1,9,1800,Gd,2,702,Ex,1800,1800,Ex,2,2007,2007,2007
1,2,6,1082,TA,1,240,TA,1082,1082,TA,1,1948,1950,1948
2,3,6,1573,Gd,2,440,Gd,756,769,Gd,2,2000,2000,2000
3,4,6,2443,Gd,3,744,Gd,1158,1158,Gd,2,2004,2004,2004
4,5,5,1040,TA,2,686,TA,1040,1040,TA,1,1968,1968,1991


In [30]:
# 예측의 재료 가져오기
test_X = test[x_feature]

# 예측을 진행
predict_test = a * test_X + b

# 예측 결과를 확인합니다
predict_test

0       221686.45
1       133257.57
2       193729.13
3       300878.33
4       128084.85
          ...    
1345    237943.57
1346    112320.37
1347    106039.21
1348    176117.25
1349    287823.37
Name: Gr Liv Area, Length: 1350, dtype: float64

In [31]:
# 제출용 sample 파일을 불러오기
submission = pd.read_csv("./housing/sample_submission.csv")
submission

,id,target
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
1345,1346,0
1346,1347,0
1347,1348,0
1348,1349,0


In [32]:
# 위에서 구한 예측값을 그대로 넣어주기
submission[y_feature] = predict_Y

# 데이터가 잘 들어갔는지 확인
submission

,id,target
0,1,294597.17
1,2,166510.77
2,3,110842.45
3,4,144588.29
4,5,241145.73
...,...,...
1345,1346,216267.41
1346,1347,338442.13
1347,1348,149514.69
1348,1349,110103.49


In [33]:
# submission을 csv 파일로 저장
# index=False란 추가적인 id를 부여할 필요가 없다는 뜻
submission.to_csv("submission.csv", index=False)